# Environment Setup

In [4]:
import nest_asyncio
import time
import random

from pydantic_ai import Agent, RunContext
from dotenv import find_dotenv, load_dotenv
from pprint import pprint

nest_asyncio.apply()
load_dotenv(find_dotenv())


True

# Function Tools

There are a number of ways to register tools with an agent:
- `@agent.tool` decorator — for tools that need access to the agent context
- `@agent.tool_plain` decorator — for tools that do not need access to the agent context
- tools keyword argument to Agent which can take either plain functions, or instances of Tool

In [ ]:


agent = Agent(
    "google-gla:gemini-1.5-flash",
    deps_type=str,
    system_prompt=(
        "You're a dice game, you should roll the die and see if the number "
        "you get back matches the user's guess. If so, tell them they're a winner. "
        "Use the player's name in the response and tell the winner number in case the player lose."
    ),
)


In [18]:
@agent.tool_plain
def roll_die() -> str:
    """Roll a six-sided die and return the result."""
    return str(random.randint(1, 6))


@agent.tool
def get_player_name(ctx: RunContext[str]) -> str:
    """Get the player's name."""
    return ctx.deps


In [19]:
for i in range(10):
    while True:
        try:
            dice_result = agent.run_sync("My guess is 4", deps="Anne")
            print(dice_result.output)
            break
        except Exception:
            pass
    

Anne, you lose! The number was 3.

Anne, you're a winner!

Anne, you lose! The number was 1.

Anne, you lose! The number was 1.

Anne, I'm sorry, but you didn't win. The number was 1.

Anne, I'm sorry, but you didn't win. The number was 5.

Anne, I'm sorry, but you didn't win. The number was 5.

Anne, I'm sorry, but you did not win. The number was 1.

Anne, unfortunately you did not win. The number was 5.

Anne, you lose! The number was 1.



In [20]:
pprint(dice_result.all_messages())

[ModelRequest(parts=[SystemPromptPart(content="You're a dice game, you should "
                                              'roll the die and see if the '
                                              'number you get back matches the '
                                              "user's guess. If so, tell them "
                                              "they're a winner. Use the "
                                              "player's name in the response "
                                              'and tell the winner number in '
                                              'case the player lose.',
                                      timestamp=datetime.datetime(2025, 4, 26, 16, 54, 59, 902581, tzinfo=datetime.timezone.utc),
                                      dynamic_ref=None,
                                      part_kind='system-prompt'),
                     UserPromptPart(content='My guess is 4',
                                    timestamp=datetime.datetime(2

# Registering Function Tools via kwarg

As well as using the decorators, we can register tools via the tools argument to the Agent constructor. This is useful when you want to reuse tools, and can also give more fine-grained control over the tools.

In [11]:
system_prompt = """\
You're a dice game, you should roll the die and see if the number
you get back matches the user's guess. If so, tell them they're a winner.
Use the player's name in the response and tell the winner number in case the player lose.
"""


In [12]:
def roll_a_dice() -> str:
    """Roll a six-sided die and return the result."""
    return str(random.randint(1, 6))


def read_player_name(ctx: RunContext[str]) -> str:
    """Get the player's name."""
    return ctx.deps


The simplest way to register tools via the Agent constructor is to pass a list of functions, the function signature is inspected to determine if the tool takes RunContext.

In [13]:
agent_a = Agent(
    "google-gla:gemini-1.5-flash",
    deps_type=str,
    tools=[roll_a_dice, read_player_name],
    system_prompt=system_prompt,
)


`agent_a` and `agent_b` are identical — but we can use Tool to reuse tool definitions and give more fine-grained control over how tools are defined, e.g. setting their name or description, or using a custom prepare method.

In [14]:
from pydantic_ai.tools import Tool

agent_b = Agent(
    "google-gla:gemini-1.5-flash",
    deps_type=str,
    tools=[
        Tool(roll_a_dice, takes_ctx=False),
        Tool(read_player_name, takes_ctx=True),
    ],
    system_prompt=system_prompt,
)


In [15]:
dice_result = {}
dice_result["a"] = agent_a.run_sync("My guess is 6", deps="Yashar")
print(dice_result["a"].output)


Hi Yashar, I rolled a 1.  Better luck next time!



Get me foobar.
{'additionalProperties': False, 'properties': {'a': {'description': 'apple pie', 'type': 'integer'}, 'b': {'description': 'banana cake', 'type': 'string'}, 'c': {'additionalProperties': {'items': {'type': 'number'}, 'type': 'array'}, 'description': 'carrot smoothie', 'type': 'object'}}, 'required': ['a', 'b', 'c'], 'type': 'object'}
Get me foobar.
{'additionalProperties': False, 'properties': {'a': {'description': 'apple pie', 'type': 'integer'}, 'b': {'description': 'banana cake', 'type': 'string'}, 'c': {'additionalProperties': {'items': {'type': 'number'}, 'type': 'array'}, 'description': 'carrot smoothie', 'type': 'object'}}, 'required': ['a', 'b', 'c'], 'type': 'object'}
Get me foobar.
{'additionalProperties': False, 'properties': {'a': {'description': 'apple pie', 'type': 'integer'}, 'b': {'description': 'banana cake', 'type': 'string'}, 'c': {'additionalProperties': {'items': {'type': 'number'}, 'type': 'array'}, 'description': 'carrot smoothie', 'type': 'object'}

In [16]:
dice_result["b"] = agent_b.run_sync("My guess is 4", deps="Anne")
print(dice_result["b"].output)

Anne, I'm sorry, but you did not win. The number was 3.



# Function Tools vs. Structured Outputs

- Function tools use the model's "tools" or "functions" API to let the model know what is available to call. 
- Tools or functions are also used to define the schema(s) for structured responses, thus a model might have access to many tools, some of which call function tools while others end the run and produce a final output.

## Function Tools and Schema

- Function parameters are extracted from the function signature, and all parameters except `RunContext` are used to build the schema for that tool call.
- PydanticAI extracts the docstring from functions using griffe and adds them to the schema. Griffe supports extracting parameter descriptions from google, numpy, and sphinx style docstrings.

In [30]:
from pydantic_ai.messages import ModelMessage, ModelResponse, TextPart
from pydantic_ai.models.function import AgentInfo, FunctionModel

agent = Agent()

In [31]:
@agent.tool_plain(docstring_format="google", require_parameter_descriptions=True)
def foobar(a: int, b: str, c: dict[str, list[float]]) -> str:
    """Get me foobar.

    Args:
        a: apple pie
        b: banana cake
        c: carrot smoothie
    """
    return f"{a} {b} {c}"


In [32]:
def print_schema(messages: list[ModelMessage], info: AgentInfo) -> ModelResponse:
    tool = info.function_tools[0]
    print(tool.description)
    print(tool.parameters_json_schema)

    return ModelResponse(parts=[TextPart("foobar")])


agent.run_sync("hello", model=FunctionModel(print_schema))


AgentRunResult(output='foobar')

In [33]:
from pydantic import BaseModel
from pydantic_ai.models.test import TestModel


agent = Agent()


class Foobar(BaseModel):
    """This is a Foobar"""

    x: int
    y: str
    z: float = 3.14


@agent.tool_plain
def foobar(f: Foobar) -> str:
    return str(f)


test_model = TestModel()
result = agent.run_sync("hello", model=test_model)
print(result.output)

{"foobar":"x=0 y='a' z=3.14"}


In [35]:
pprint(test_model.last_model_request_parameters.function_tools, indent=2)

[ ToolDefinition(name='foobar',
                 description='This is a Foobar',
                 parameters_json_schema={ 'properties': { 'x': { 'type': 'integer'},
                                                          'y': { 'type': 'string'},
                                                          'z': { 'default': 3.14,
                                                                 'type': 'number'}},
                                          'required': ['x', 'y'],
                                          'title': 'Foobar',
                                          'type': 'object'},
                 outer_typed_dict_key=None,
                 strict=None)]


## Dynamic Function Tools

In [36]:
from typing import Union

from pydantic_ai import Agent, RunContext
from pydantic_ai.tools import ToolDefinition

agent = Agent('test')


async def only_if_42(
    ctx: RunContext[int], tool_def: ToolDefinition
) -> Union[ToolDefinition, None]:
    if ctx.deps == 42:
        return tool_def


@agent.tool(prepare=only_if_42)
def hitchhiker(ctx: RunContext[int], answer: str) -> str:
    return f'{ctx.deps} {answer}'

In [37]:
result = agent.run_sync("testing...", deps=41)
print(result.output)

success (no tool calls)


In [38]:
result = agent.run_sync("testing...", deps=42)
print(result.output)

{"hitchhiker":"42 a"}


In [40]:
from __future__ import annotations

from typing import Literal

from pydantic_ai import Agent, RunContext
from pydantic_ai.models.test import TestModel
from pydantic_ai.tools import Tool, ToolDefinition


def greet(name: str) -> str:
    return f"hello {name}"


async def prepare_greet(
    ctx: RunContext[Literal["human", "machine"]], tool_def: ToolDefinition
) -> ToolDefinition | None:
    d = f"Name of the {ctx.deps} to greet."
    tool_def.parameters_json_schema["properties"]["name"]["description"] = d
    return tool_def


greet_tool = Tool(greet, prepare=prepare_greet)
test_model = TestModel()
agent = Agent(test_model, tools=[greet_tool], deps_type=Literal["human", "machine"])

result = agent.run_sync("testing...", deps="human")
print(result.output)



{"greet":"hello a"}


In [42]:
pprint(test_model.last_model_request_parameters.function_tools, indent=2)


[ ToolDefinition(name='greet',
                 description='',
                 parameters_json_schema={ 'additionalProperties': False,
                                          'properties': { 'name': { 'description': 'Name '
                                                                                   'of '
                                                                                   'the '
                                                                                   'human '
                                                                                   'to '
                                                                                   'greet.',
                                                                    'type': 'string'}},
                                          'required': ['name'],
                                          'type': 'object'},
                 outer_typed_dict_key=None,
                 strict=None)]
